In [4]:
import requests
import pandas as pd
import os
import folium
from dotenv import load_dotenv
%matplotlib widget


# Load environment variables
load_dotenv()

# Access the API key
api_key = os.getenv('API_KEY_FOR_EV_STATION')
base_url = "https://developer.nrel.gov/api/alt-fuel-stations/v1.json"

# Bronx zip codes
bronx_zipcode = [
    10451, 10452, 10453, 10454, 10455, 10456, 10457, 10458, 10459, 10460,
    10461, 10462, 10463, 10464, 10465, 10466, 10467, 10468, 10469, 10470,
    10471, 10472, 10473, 10474, 10475
]

# Queens zip codes
queens_zipcode = [
    11001, 11004, 11005, 11101, 11102, 11103, 11104, 11105, 11106, 11109,
    11354, 11355, 11356, 11357, 11358, 11360, 11361, 11362, 11363, 11364,
    11365, 11366, 11367, 11368, 11369, 11370, 11372, 11373, 11374, 11375,
    11377, 11378, 11379, 11385, 11411, 11412, 11413, 11414, 11415, 11416,
    11417, 11418, 11419, 11420, 11421, 11422, 11423, 11426, 11427, 11428,
    11429, 11432, 11433, 11434, 11435, 11436, 11451, 11691, 11692, 11693,
    11694, 11695, 11697
]

# Brooklyn zip codes
brooklyn_zipcode = [
    11201, 11203, 11204, 11205, 11206, 11207, 11208, 11209, 11210, 11211, 
    11212, 11213, 11214, 11215, 11216, 11217, 11218, 11219, 11220, 11221, 
    11222, 11223, 11224, 11225, 11226, 11228, 11229, 11230, 11231, 11232, 
    11233, 11234, 11235, 11236, 11237, 11238, 11239
]

# Staten Island zip codes
staten_island_zipcode = [
    10301, 10302, 10303, 10304, 10305, 10306, 10307, 10308, 10309, 10310, 
    10312, 10314
]

# Manhattan zip codes
manhattan_zipcode = [
    10001, 10002, 10003, 10004, 10005, 10006, 10007, 10009, 10010, 10011, 
    10012, 10013, 10014, 10016, 10017, 10018, 10019, 10020, 10021, 10022, 
    10023, 10024, 10025, 10026, 10027, 10028, 10029, 10030, 10031, 10032, 
    10033, 10034, 10035, 10036, 10037, 10038, 10039, 10040, 10044, 10065, 
    10069, 10075, 10128, 10280, 10282
]

# Combine all lists
all_zip_codes = (
    [str(zip_code) for zip_code in bronx_zipcode] + 
    [str(zip_code) for zip_code in queens_zipcode] + 
    [str(zip_code) for zip_code in brooklyn_zipcode] + 
    [str(zip_code) for zip_code in staten_island_zipcode] + 
    [str(zip_code) for zip_code in manhattan_zipcode]
)

def fetch_data_for_zip_codes(zip_codes):
    results = []

    for zip_code in zip_codes:
        params = {
            'api_key': api_key,
            'state': 'NY',
            'zip': zip_code,
            'limit': 50  # Adjust limit as necessary
        }
        response = requests.get(base_url, params=params)

        if response.status_code == 200:
            json_data = response.json()
            stations = json_data['fuel_stations']
            for station in stations:
                results.append({
                    'zip_code': zip_code,
                    'city': station.get('city'),
                    'latitude': station.get('latitude'),
                    'longitude': station.get('longitude'),
                    'total_results': json_data['total_results']
                })
        else:
            print(f"Failed to fetch data for ZIP code {zip_code}: Status code {response.status_code}")
        
    return results

data = fetch_data_for_zip_codes(all_zip_codes)
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_filename = 'NY_EV_Stations.csv'
df.to_csv(csv_filename, index=False)

# Create a map centered around New York
ny_map = folium.Map(location=[40.730610, -73.935242], zoom_start=12)

# Add markers to the map
for idx, row in df.iterrows():
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        folium.Marker([row['latitude'], row['longitude']],
                      popup=f"City: {row['city']}, ZIP: {row['zip_code']}").add_to(ny_map)

# Display the map
ny_map.save('NY_EV_Stations.html')
ny_map






In [16]:
# Create a download link for the CSV file (for Jupyter notebooks)
def create_download_link(filename):
    import base64
    from IPython.display import HTML

    with open(filename, 'rb') as f:
        data = f.read()
    b64 = base64.b64encode(data).decode()
    return HTML(f'<a download="{filename}" href="data:application/octet-stream;base64,{b64}" target="_blank">Download {filename}</a>')

create_download_link(csv_filename)

In [17]:
df

,zip_code,city,latitude,longitude,total_results
0,10451,Bronx,40.822142,-73.931128,1
1,10454,Bronx,40.807338,-73.930047,1
2,10455,Bronx,40.808151,-73.902589,2
3,10455,Bronx,40.805490,-73.900490,2
4,10458,Bronx,40.860950,-73.889379,2
...,...,...,...,...,...
563,10282,New York,40.773730,-73.989713,5
564,10282,New York,40.716950,-74.014948,5
565,10282,New York,40.716131,-74.015548,5
566,10282,New York,40.716132,-74.015557,5


In [10]:
from folium.plugins import HeatMap

# Load the CSV file
file_path = './NY_EV_Stations.csv'
data = pd.read_csv(file_path)

# Create a base map centered around New York City
map_center = [40.7128, -74.0060]
base_map = folium.Map(location=map_center, zoom_start=11)

# Extract latitude and longitude data
heat_data = data[['latitude', 'longitude']].values.tolist()


# Add a heatmap layer to the base map
HeatMap(heat_data).add_to(base_map)

# Save the map to an HTML file
map_output_path = './NY_EV_Stations_Heatmap.html'
base_map.save(map_output_path)

print(f"Heatmap saved to {map_output_path}")


Heatmap saved to ./NY_EV_Stations_Heatmap.html
